In [66]:
max_morphemes = 10
PAD = 0

In [67]:
import pickle

try:
    with open('crawling/news.pickle', 'rb') as f:
        articles = pickle.load(f)
except FileNotFoundError:
    with open('../crawling/news.pickle', 'rb') as f:
        articles = pickle.load(f)


In [68]:
bodies = articles['body'].to_list()

In [69]:
import re

sentences = []
for body in bodies:
    for sentence in body.split('.'):
        if not re.match('.*[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9 ].*', sentence):
            sentences.append(sentence)

In [70]:
from konlpy.tag import Okt

okt = Okt()

data = [(okt.nouns(sentence), okt.morphs(sentence)) for sentence in sentences]
data = [(keywords, sentence) for keywords, sentence in data
        if 0 < len(keywords) <= max_morphemes]

data_keywords = [keywords for keywords, _ in data]
data_sentences = [sentence for _, sentence in data]

num_sentence_morphs = max([len(sentence) for sentence in data_sentences])

In [71]:
def remove_overlaps(keywords):
    result = []
    for keyword in keywords:
        if keyword not in result:
            result.append(keyword)
    return result

data_keywords = [remove_overlaps(keywords) for keywords in data_keywords]

In [72]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_sentences)
tokenizer.index_word[PAD] = PAD

num_of_morphemes = len(tokenizer.word_counts) + 1

keywords_sequences = tokenizer.texts_to_sequences(data_keywords)
sentence_sequences = tokenizer.texts_to_sequences(data_sentences)

In [73]:
def padding(sequence, max_len):
    return sequence + [PAD] * (max_len - len(sequence))

keywords_sequences = [padding(seq, max_morphemes) for seq in keywords_sequences]
sentence_sequences = [padding(seq, max_morphemes) for seq in sentence_sequences]

In [74]:
import numpy as np

keywords = np.array(keywords_sequences)
sentences = np.array(sentence_sequences)
